In [15]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os

#### Import stations_sorted and keep top n

In [52]:
stations_sorted = pd.read_csv('../../data/parameters/stations_sorted.csv')
top_n=stations_sorted[:30]

#### Import station_information and keep only top n

In [53]:
station_information = pd.read_csv('../../data/stations/station_information.csv')
station_information_topn=station_information.merge(top_n,on='station_id')

#### Create match between stations_id and rows in Optim model

In [54]:
def create_stations_match(stations_sorted):
    stations_match = dict()
    for i in range(1,len(stations_sorted)+1):
        stations_match[i] = stations_sorted.iloc[i-1].station_id
    return stations_match

In [73]:
stations_match=create_stations_match(stations_sorted)
row_to_station_id=create_stations_match(top_n)
station_id_to_row = {v: k for k, v in row_to_station_id.items()}
# NB : row numbers start at 1 cf Julia

## Load results

In [56]:
# folder should be named 'nb_stations'_'nb_vans'_'S'_*
def load_results(data_folder):
    results_folder = os.path.join("../../results/data", data_folder)
    nb_stations = int(data_folder.split('_')[2])
    nb_vans = int(data_folder.split('_')[4])

    with open(os.path.join(results_folder, 'x.csv'), 'r') as f:
        x = f.read().splitlines()
    x = np.array(x).astype(float).reshape((nb_stations,nb_stations,nb_vans,24),order='F')

    with open(os.path.join(results_folder, 'v.csv'), 'r') as f:
        v = f.read().splitlines()
    v = np.array(v).astype(float).reshape((nb_stations,nb_stations,nb_vans,24),order='F')

    with open(os.path.join(results_folder, 'z.csv'), 'r') as f:
        z = f.read().splitlines()
    z = np.array(z).astype(float).reshape((nb_stations,nb_stations,nb_vans,24),order='F')

    with open(os.path.join(results_folder, 'y.csv'), 'r') as f:
        y = f.read().splitlines()
    y = np.array(y).astype(float).reshape((nb_stations,24),order='F')

    with open(os.path.join(results_folder, 'w.csv'), 'r') as f:
        w = f.read().splitlines()
    w = np.array(w).astype(float).reshape((nb_stations,nb_stations,24),order='F')

    with open(os.path.join(results_folder, 'u.csv'), 'r') as f:
        u = f.read().splitlines()
    u = np.array(u).astype(float).reshape((nb_stations,nb_stations,24),order='F')

    with open(os.path.join(results_folder, 'd.csv'), 'r') as f:
        d = f.read().splitlines()
    d = np.array(d).astype(float).reshape((nb_stations,nb_stations,24),order='F')

    return x, v, z, y, w, u, d

In [57]:
x, v, z, y, w, u, d = load_results('2022-12-05T002617_nstations_31_nvans_2_capacity_25_lambda_15_timelimit_30')

# Plot stations

In [58]:
fig = px.scatter_mapbox(
    station_information_topn, 
    lat="lat", 
    lon="lon", 
    height=800,
    width=800)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

# Plot demand

In [59]:
def plot_demand(d):
    fig = go.Figure()
    demands = np.array([d[station_id_to_row[station_id],:,:].sum() for station_id in station_information_topn['station_id']])
    demands_scaled = 25*demands/np.max(demands)
    fig.add_trace(go.Scattermapbox(
        lat=station_information_topn['lat'],
        lon=station_information_topn['lon'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=demands_scaled,
        )))
    fig.update_layout(
        mapbox_style="open-street-map",
        hovermode='closest',
        mapbox=dict(
            bearing=0,
            center=go.layout.mapbox.Center(
                lat=42.364981,
                lon=-71.079021,
            ),
            pitch=0,
            zoom=11
        ),
        width=800,
        height=800,
    )
    fig.show()
plot_demand(d)


# Plots rebalancing trips

In [61]:
i,j=9,21
[station_information.loc[station_information['station_id']==row_to_station_id[i],"lon"].iloc[0],station_information.loc[station_information['station_id']==row_to_station_id[j],"lon"].iloc[0]]

[-71.07404083013535, -71.07942931372236]

In [70]:
for t in range(24):
    if x[:,:,0,t].sum()>0:
        i,j = np.argwhere(x[:,:,0,t] != 0)[0]
        #print(i,j)

In [63]:
def plot_trips(v,z,van_id):
    fig = go.Figure()
    
    fig.add_trace(go.Scattermapbox(
        lat=station_information_topn['lat'],
        lon=station_information_topn['lon'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=8,
        )))

    # plot trips
    for t in range(24):
        if z[:,:,van_id,t].sum()>0:
            i,j = np.argwhere(z[:,:,van_id,t] != 0)[0]
            fig.add_trace(go.Scattermapbox(
                mode="lines",
                lon=[station_information.loc[station_information['station_id']==
                        row_to_station_id[i],"lon"].iloc[0],
                     station_information.loc[station_information['station_id']==
                                             row_to_station_id[j],"lon"].iloc[0]],
                lat=[station_information.loc[station_information['station_id']==row_to_station_id[i],"lat"].iloc[0],station_information.loc[station_information['station_id']==row_to_station_id[j],"lat"].iloc[0]],
                line=dict(width=z[i,j,van_id,t]/5, color='red'),
                showlegend=True
            ))
        else:
            pass
        i,j = np.argwhere(v[:,:,van_id,t] != 0)[0]
        if i!=j:
            fig.add_trace(go.Scattermapbox(
                mode="lines",
                lon=[station_information.loc[station_information['station_id']==row_to_station_id[i],"lon"].iloc[0],station_information.loc[station_information['station_id']==row_to_station_id[j],"lon"].iloc[0]],
                lat=[station_information.loc[station_information['station_id']==row_to_station_id[i],"lat"].iloc[0],station_information.loc[station_information['station_id']==row_to_station_id[j],"lat"].iloc[0]],
                line=dict(width=1, color='blue'),
                showlegend=False
            ))

    fig.update_layout(
        # mapbox_style="open-street-map",
        mapbox_style="carto-positron",
        hovermode='closest',
        # margin={"r":0,"t":0,"l":0,"b":0},
        mapbox=dict(
            bearing=0,
            center=go.layout.mapbox.Center(
                lat=42.364981,
                lon=-71.079021,
            ),
            pitch=0,
            zoom=11
        ),
        width=800,
        height=800,
        showlegend=True
    )
    fig.show()
plot_trips(v,z,0)

KeyError: 0

In [71]:
# load results 
results = pd.read_csv('../data/sequential_model_day1/results.csv') 
results.columns = ["start","end","van","hour","bikes"]

In [74]:
# see itinerary of van 1
results_van1 = results[results['van']==1]
results_van1["start"] = results_van1["start"].map(stations_match)
results_van1["end"] = results_van1["end"].map(stations_match)
results_van1["start_lat"] = results_van1["start"].map(dict(zip(station_information["station_id"],station_information["lat"])))
results_van1["start_lon"] = results_van1["start"].map(dict(zip(station_information["station_id"],station_information["lon"])))
results_van1["end_lat"] = results_van1["end"].map(dict(zip(station_information["station_id"],station_information["lat"])))
results_van1["end_lon"] = results_van1["end"].map(dict(zip(station_information["station_id"],station_information["lon"])))
results_van1 = results_van1.reset_index(drop=True)

In [75]:
results_van1.sort_values(by=['hour','bikes'], ascending=[True,False])

,start,end,van,hour,bikes,start_lat,start_lon,end_lat,end_lon
5,45,46,1.0,1.0,0.0,NaN,NaN,42.343666,-71.085824
6,45,46,1.0,2.0,0.0,NaN,NaN,42.343666,-71.085824
7,45,46,1.0,3.0,0.0,NaN,NaN,42.343666,-71.085824
8,45,46,1.0,4.0,0.0,NaN,NaN,42.343666,-71.085824
9,45,46,1.0,5.0,0.0,NaN,NaN,42.343666,-71.085824
10,45,46,1.0,6.0,0.0,NaN,NaN,42.343666,-71.085824
11,45,46,1.0,7.0,0.0,NaN,NaN,42.343666,-71.085824
12,45,46,1.0,8.0,0.0,NaN,NaN,42.343666,-71.085824
13,45,46,1.0,9.0,0.0,NaN,NaN,42.343666,-71.085824
14,45,46,1.0,10.0,0.0,NaN,NaN,42.343666,-71.085824


In [76]:
# load optimal results
results_day = results.groupby(["start","end"])["bikes"].sum().reset_index()
results_day["start"] = results_day["start"].map(stations_match)
results_day["end"] = results_day["end"].map(stations_match)
results_day["start_lat"] = results_day["start"].map(dict(zip(station_information["station_id"],station_information["lat"])))
results_day["start_lon"] = results_day["start"].map(dict(zip(station_information["station_id"],station_information["lon"])))
results_day["end_lat"] = results_day["end"].map(dict(zip(station_information["station_id"],station_information["lat"])))
results_day["end_lon"] = results_day["end"].map(dict(zip(station_information["station_id"],station_information["lon"])))
results_day.dropna(inplace=True)
results_day = results_day.reset_index(drop=True)


In [77]:
import plotly.graph_objects as go 

fig = go.Figure(go.Scattermapbox(
    lat=station_information['lat'],
    lon=station_information['lon'],
    mode='markers',
    marker=go.scattermapbox.Marker(
        size=9
    ),
))

fig.update_layout(
    autosize=True,
    mapbox_style="carto-positron",
    hovermode='closest',
    mapbox=dict(
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=42.364981,
            lon=-71.079021,
        ),
        pitch=0,
        zoom=10
    ),
    height=600,
    width=600
)

for i in range(len(results_van1)):
    fig.add_trace(
        go.Scattermapbox(
            mode="lines",
            lon=[results_van1.loc[i,"start_lon"],results_van1.loc[i,"end_lon"]],
            lat=[results_van1.loc[i,"start_lat"],results_van1.loc[i,"end_lat"]],
            line=dict(width=results_van1.loc[i,"bikes"]/5, color='red'),
        )
    )
    
fig.show()